In [1]:
import pandas as pd
import numpy as np

import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from scipy.stats import chi2_contingency

import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

In [2]:
# read data:
data_dir = 'Data/train.csv'
data = pd.read_csv(data_dir, header = 0);
display(data.head(5))
data.drop(columns=['Name', 'PassengerId'], inplace=True)
display(data.shape)

PassengerId HomePlanet CryoSleep  Cabin  Destination   Age    VIP  \
0     0001_01     Europa     False  B/0/P  TRAPPIST-1e  39.0  False   
1     0002_01      Earth     False  F/0/S  TRAPPIST-1e  24.0  False   
2     0003_01     Europa     False  A/0/S  TRAPPIST-1e  58.0   True   
3     0003_02     Europa     False  A/0/S  TRAPPIST-1e  33.0  False   
4     0004_01      Earth     False  F/1/S  TRAPPIST-1e  16.0  False   

   RoomService  FoodCourt  ShoppingMall     Spa  VRDeck               Name  \
0          0.0        0.0           0.0     0.0     0.0    Maham Ofracculy   
1        109.0        9.0          25.0   549.0    44.0       Juanna Vines   
2         43.0     3576.0           0.0  6715.0    49.0      Altark Susent   
3          0.0     1283.0         371.0  3329.0   193.0       Solam Susent   
4        303.0       70.0         151.0   565.0     2.0  Willy Santantines   

   Transported  
0        False  
1         True  
2        False  
3        False  
4         True

(8693, 12)

In [3]:
display(pd.DataFrame(data.count()).transpose())
data.dropna(axis=0, inplace=True)
display(pd.DataFrame(data.count()).transpose())

HomePlanet  CryoSleep  Cabin  Destination   Age   VIP  RoomService  \
0        8492       8476   8494         8511  8514  8490         8512   

   FoodCourt  ShoppingMall   Spa  VRDeck  Transported  
0       8510          8485  8510    8505         8693

HomePlanet  CryoSleep  Cabin  Destination   Age   VIP  RoomService  \
0        6764       6764   6764         6764  6764  6764         6764   

   FoodCourt  ShoppingMall   Spa  VRDeck  Transported  
0       6764          6764  6764    6764         6764

In [4]:
# break Cabin into three categorical variables:
data['CabinDeck'] = data['Cabin'].str.extract(r'^(\S)/.*')
data['CabinNum'] = data['Cabin'].str.extract(r'.*/(\d+)/.*')
data['CabinSide'] = data['Cabin'].str.extract(r'.*/(\S)$')

categorical_columns = ['HomePlanet', 'CryoSleep', 'Destination', 'CabinDeck', 'CabinNum', 'CabinSide', 'VIP']

for var in categorical_columns:
    crosstab = pd.crosstab(data[var], data['Transported']).transpose()
    display(crosstab)
    display(chi2_contingency(crosstab).pvalue)
    

HomePlanet   Earth  Europa  Mars
Transported                     
False         2101     585   677
True          1555    1126   720

8.050363466197175e-56

CryoSleep    False  True 
Transported              
False         2925    438
True          1459   1942

0.0

Destination  55 Cancri e  PSO J318.5-22  TRAPPIST-1e
Transported                                         
False                551            317         2495
True                 891            323         2187

...

0.06742886228130271

CabinSide       P     S
Transported            
False        1840  1523
True         1499  1902

2.6585801469085895e-18

VIP          False  True 
Transported              
False         3261    102
True          3339     62

0.0016001674726641246

In [5]:
# generate scatter plots and histgrams:
fig, ax = plt.subplots(5,1,  figsize=(10, 10))
plt.subplots_adjust(top = 2)

for i, var in enumerate(['Age', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']):
    if var != 'Age':
        data[var] = np.log(data[var] - data[var].min() + 1e-10)
    
    sns.histplot(data[[var, 'Transported']], x=var, hue='Transported', bins=50, ax=ax[i],)


In [6]:
# data = pd.get_dummies(data, columns=['HomePlanet', 'CryoSleep', 'CabinDeck', 'CabinSide', 'Destination', 'VIP'], drop_first=True)
data.drop(columns=['Cabin', 'CabinNum'], inplace=True);
label_encoder = LabelEncoder()

for var in categorical_columns:
    if var in data:
        data[var] = label_encoder.fit_transform(data[var])

In [7]:
# train random forest model:

X_train, X_test, y_train, y_test = train_test_split(
    data.drop(columns=['Transported']),
    data['Transported'],
    test_size=0.2,
    random_state=42
)

# Define the hyperparameter grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Initialize Random Forest classifier
rf_classifier = RandomForestClassifier(random_state=42)

# Create GridSearchCV object
grid_search = GridSearchCV(estimator=rf_classifier, param_grid=param_grid, cv=3, scoring='accuracy')

# Perform grid search on the training data
grid_search.fit(X_train, y_train)

# Print the best hyperparameters found
best_params = grid_search.best_params_
print(f"Best Hyperparameters: {best_params}")

# Make predictions on the test set using the best model
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# Evaluate the best model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Display classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))


Best Hyperparameters: {'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 50}
Accuracy: 0.81
Classification Report:
              precision    recall  f1-score   support

       False       0.82      0.81      0.81       699
        True       0.80      0.81      0.80       654

    accuracy                           0.81      1353
   macro avg       0.81      0.81      0.81      1353
weighted avg       0.81      0.81      0.81      1353



In [23]:

def preprocess(data):
    data.drop(columns=['Name', 'PassengerId'], inplace=True)
    # data.dropna(axis=0, inplace=True)

    # break Cabin into three categorical variables:
    data['CabinDeck'] = data['Cabin'].str.extract(r'^(\S)/.*')
    data['CabinNum'] = data['Cabin'].str.extract(r'.*/(\d+)/.*')
    data['CabinSide'] = data['Cabin'].str.extract(r'.*/(\S)$')

    for var in categorical_columns:
        data[var] = label_encoder.fit_transform(data[var])

    for var in ['FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']:
        data[var] = np.log(data[var] - data[var].min() + 1e-10)

    # data = pd.get_dummies(data, columns=['HomePlanet', 'CryoSleep', 'CabinDeck', 'CabinSide', 'Destination', 'VIP'], drop_first=True)
    data.drop(columns=['Cabin', 'CabinNum'], inplace=True)
    x = data.fillna(0)

    return x

data_test = pd.read_csv('Data/test.csv', header=0)
passenger = data_test['PassengerId'];
x_test = preprocess(data_test)
y_pred = best_model.predict(x_test)



In [24]:
data_prediction = pd.DataFrame({'PassengerId': passenger, 'Transported': y_pred})

In [29]:
data_prediction.to_csv('Data/predictions.csv', index=None)